# Extracting BERT embeddings from the captions

In this notebook, I attempt to extract BERT word embeddings from the captions. I use the steps outlined in section 4.4 of [this](https://www.overleaf.com/project/5ca66278f4224d0690dd9e29) source to achieve this.
I follow [this](https://mccormickml.com/2019/05/14/BERT-word-embeddings-tutorial/) tutorial.

More sources consulted:
 - http://mccormickml.com/2019/07/22/BERT-fine-tuning/
 - [Huggingface's much easier one](https://huggingface.co/bert-base-uncased?text=The+goal+of+life+is+%5BMASK%5D.)

# TODO:
 - use TFBert and batch tokenize
 - detokenize using wordpiece perhaps?

## Setup

In [232]:
## imports 
import torch
from transformers import BertTokenizer,BertModel
from tqdm import tqdm
#from pytorch_pretrained_bert import BertTokenizer
#from pytorch_pretrained_bert import BertModel

In [57]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2060


In [58]:
import logging
logging.basicConfig(level=logging.INFO)

In [59]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

## Trying out with one sentence

In [60]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = "Here is the sentence I want embeddings for."
encoded_input = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=50)
output = model(**encoded_input)

In [61]:
print(encoded_input)
print(output.pooler_output.size())
print(output.last_hidden_state.size())

{'input_ids': tensor([[ 101, 2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005,
         1012,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])}
torch.Size([1, 768])
torch.Size([1, 50, 768])


In [62]:
#trying on a single sentence
# Print the original sentence.
sentence = "Here is the sentence I want embeddings for."
print(' Original: ', sentence)

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentence))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentence)))


 Original:  Here is the sentence I want embeddings for.
Tokenized:  ['here', 'is', 'the', 'sentence', 'i', 'want', 'em', '##bed', '##ding', '##s', 'for', '.']
Token IDs:  [2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005, 1012]


## No detokenization: word tokens used as-is by bert (multiple sentences, not vectorized)

In [83]:
#trying on a list of sentences:

sentences = [
    "Here is the sentence I want embeddings for.",
    "Our friends won't buy this analysis, let alone the next one we propose.",
    "snowman on a red background",
    "celebrity celebrates with his team mates after the second goal was scored by football player , during the match .",
    "action shot of basketball player , passing the ball during a game .",
    "a sunny winter day by lake .",
    "the extraordinary domed ceiling , with an artist 's hand - painted mural of delicate clouds , creates a vaulted open and airy feel in the space ."
      
]

In [ ]:
def bert_tokenize(max_len, sentences, tokenizer):
    """
    Takes in a maximum length and a list of sentences and uses bert to tokenize them.
    """
    input_ids = []
    attention_masks = []
    for sent in sentences:
        encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len
                        return_tensors = 'pt',     # Return pytorch tensors.
                        padding = 'max_length',
                        truncation = True
                   )
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
        
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])
        
    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    

    # Print sentence 0, now as a list of IDs.
    print('Original: ', sentences[0])
    print('Token IDs:', input_ids[0])
    # Display the words with their indeces.
    for tok_id in input_ids[0]:
        token = list(tokenizer.vocab.values()).index(tok_id.item())
        print(list(tokenizer.vocab.keys())[token])
        
    
    return torch.unsqueeze(input_ids, 1), attention_masks

        

In [85]:
input_ids, atttention_masks = bert_tokenize(50, sentences, tokenizer)

Original:  Here is the sentence I want embeddings for.
Token IDs: tensor([ 101, 2182, 2003, 1996, 6251, 1045, 2215, 7861, 8270, 4667, 2015, 2005,
        1012,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0])
[CLS]
here
is
the
sentence
i
want
em
##bed
##ding
##s
for
.
[SEP]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]
[PAD]


In [66]:
sentence_num = 6
print('Original: ', sentences[sentence_num], len(sentences[sentence_num]))
print('Token IDs:', input_ids[sentence_num], len(input_ids[sentence_num]))

Original:  the extraordinary domed ceiling , with an artist 's hand - painted mural of delicate clouds , creates a vaulted open and airy feel in the space . 145
Token IDs: tensor([[  101,  1996,  9313, 29208,  5894,  1010,  2007,  2019,  3063,  1005,
          1055,  2192,  1011,  4993, 15533,  1997, 10059,  8044,  1010,  9005,
          1037, 24616,  2330,  1998,  2250,  2100,  2514,  1999,  1996,  2686,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]]) 1


In [86]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [97]:
def bert_embed(input_ids, model):
    """
    Given a list of input ids, outputs a numpy array of embeddings: 768 dimensional, padded to max_len of 50.
    TODO: disregard embedding of CLS token
    """
    input_embeddings = []
    for tokens_tensor in input_ids:
        with torch.no_grad():
            encoded_layers = model(tokens_tensor,output_hidden_states=True)
            bert_embedding = encoded_layers.last_hidden_state[0]
            input_embeddings.append(bert_embedding)
    embeddings = torch.stack(input_embeddings)
    return embeddings        
            

        
    
    

In [100]:
embeddings = bert_embed(input_ids, model)
print(embeddings.size())

torch.Size([7, 50, 768])
tensor([[-0.5574,  0.1864,  0.3395,  ..., -0.3215,  0.5786, -0.3520],
        [-1.3216, -0.2782,  0.3291,  ..., -0.0920,  0.3318,  0.1545],
        [-0.5194, -0.2498,  0.0847,  ...,  0.3952,  0.2744,  0.4622],
        ...,
        [ 0.4907, -0.1928,  0.8916,  ..., -0.6989, -0.0364, -1.2576],
        [ 0.5639, -0.2853,  0.8997,  ..., -0.7365, -0.0260, -1.2620],
        [ 0.6167, -0.2604,  0.7293,  ..., -0.7552, -0.1310, -1.5502]])


## Batch Embedding (so its faster)

In [118]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [180]:
tokenizer

PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [308]:
def bert_tokenize_batch(max_len, sentences, tokenizer):
    """
    Takes in a maximum length and a list of sentences and uses bert to tokenize them.
    """
   
    encoded_dict = tokenizer(
                    sentences,                      # Sentence to encode.
                    add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                    max_length = max_len,           # Pad & truncate all sentences.
                    padding = 'max_length',
                    truncation = True,
                    #can add back and refer to non batched version on how to retrieve attention masks
                    #return_attention_mask = True,   # Construct attn. masks.
                    return_tensors = 'pt',     # Return pytorch tensors.
               )
    
    input_ids = encoded_dict['input_ids']
    #tokenized_sentences = input_ids.apply_(lambda word: )

    

    
#test
#     print('Original: ', sentences[0])
#     print('Token IDs:', input_ids[0])
    #how to convert IDs to #seperated tokens
    #print(tokenizer.convert_ids_to_tokens(input_ids))
    # Display the words with their indeces.
#     for tok_id in input_ids[0]:
#         print(word_ids()[tok_id])
# #         token = list(tokenizer.vocab.values()).index(tok_id.item())
# #         print(list(tokenizer.vocab.keys())[token])
        
    #print(input_ids.size())
    return input_ids

In [311]:
def bert_embed_batch(input_ids, model):
    """
    Given a list of input ids, outputs a numpy array of embeddings: 768 dimensional, padded to max_len of 50.
    TODO: disregard embedding of CLS token
    """
    with torch.no_grad():
        encoded_layers = model(input_ids,output_hidden_states=True)
        bert_embedding = encoded_layers.last_hidden_state
        #print(bert_embedding.size())
    return bert_embedding
            

In [291]:
tokenized_sentences = bert_tokenize_batch(50, sentences, tokenizer)

torch.Size([7, 50])


In [292]:
embedded_sentences = bert_embed_batch(tokenized_sentences,model)

torch.Size([7, 50, 768])


## Pipeline: folder with numpy files for ecery images caption by number 

There is no need to tokenize and save tokenized captions. This can be done on the fly as we are using a fixed pretrained tokenizer from bert. We will save the embeddings, then tokenize using the same tokenizer during training/testing time so we can match the sequence lengths of the embeddings with the captions. I think this will preserve information compared to the strategy of averaging the embeddings of each subword belonging to the same word to preserve sequence length :)

In [326]:
import os
import pandas as pd
import numpy as np
data_dir = r'../data/30k_sample/'
meta_file = r'prepped_meta'

In [294]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [295]:
meta_path = os.path.join(data_dir,meta_file)

In [296]:
meta_df = pd.read_csv(meta_path)
meta_df.iloc[0]

global_index                                                         552309
caption                   drum and drumsticks pattern repeat seamless in...
link                      https://thumb7.shutterstock.com/display_pic_wi...
objects                   font,line,black-and-white,pattern,design,illus...
mid                       /m/03gq5hm,/m/03scnj,/m/01g6gs,/m/0hwky,/m/02c...
object_confidences        0.920914888381958,0.8690993189811707,0.7811979...
image_path                                data\30k_sample\images\552309.png
size                                                             (450, 470)
height                                                                  470
width                                                                   450
aspect_ratio                                                       0.957447
num_obj                                                                  11
nltk_sent_polarity        {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...
nltk_sent   

In [297]:
captions = meta_df['caption'].tolist()
global_indices = meta_df['global_index'].tolist()

In [298]:
#zip the captions and the indices of the images they belong to together so we can process and save them accordingly
caption_ids = list(zip(captions,global_indices))

In [355]:
caption_ids[0]

('drum and drumsticks pattern repeat seamless in black color for any design .',
 552309)

In [299]:
len(captions)


29928

In [323]:
num_trying = 50

In [324]:
from torch.utils.data import DataLoader

dataloader = DataLoader(caption_ids[:num_trying], batch_size=32, shuffle=False)

In [327]:
print(len(dataloader))

2


In [331]:
embeddings_dir = r'bert_embeddings'
embeddings_path = os.path.join(data_dir,embeddings_dir)
if not os.path.exists(embeddings_path):
    os.mkdir(embeddings_path)
embeddings_path

'../data/30k_sample/bert_embeddings'

In [328]:
cwd = os.getcwd()

In [329]:
cwd

'C:\\Users\\apra\\Desktop\\SPRING 2021\\DS 4400\\ML1_Final_Proj\\BERT_stuff'

In [340]:
#os.chdir(embeddings_path)
for batch_idx, batch in tqdm(enumerate(dataloader), total= len(dataloader),leave=False):
    captions,global_indices = batch
    tokenized_captions = bert_tokenize_batch(50, captions, tokenizer)
    embedded_captions = bert_embed_batch(tokenized_captions,model)
    
    
    for idx, global_idx in enumerate(global_indices):
        save_path = str(int(global_idx))
        emb = embedded_captions[idx]
        np.save(save_path,emb.numpy())
            
        
        

In [335]:
os.getcwd()

'C:\\Users\\apra\\Desktop\\SPRING 2021\\DS 4400\\ML1_Final_Proj\\data\\30k_sample\\bert_embeddings'

In [ ]:
935*32

In [352]:
tokenized_captions = bert_tokenize_batch(50, captions[:1], tokenizer)

In [353]:
embedded_captions = bert_embed_batch(tokenized_captions,model)

In [354]:
embedded_captions

tensor([[[-0.7533, -0.0418,  0.3216,  ...,  0.1586,  0.7584, -0.3688],
         [-0.2383, -0.3724,  0.2092,  ...,  0.2965,  0.7036,  0.3190],
         [-1.5021, -0.5917,  0.4151,  ...,  0.1487, -0.0201, -0.3873],
         ...,
         [ 0.4058, -0.1902,  0.9161,  ..., -0.5628,  0.2021, -1.2453],
         [ 0.4850, -0.3266,  0.9142,  ..., -0.5470,  0.2161, -1.1589],
         [ 0.6672, -0.1215,  0.6078,  ..., -0.6456,  0.1557, -1.6077]]])

## Some code to read the bert embedding files

In [343]:
emb = np.load(r'552309.npy')
emb

array([[-6.7797875e-01,  6.6109024e-02,  2.0386510e-01, ...,
        -7.0361063e-02,  6.2120736e-01, -6.3508731e-01],
       [ 3.6984134e-01,  2.8702578e-01,  3.6180964e-01, ...,
         1.9765865e-02,  4.4717944e-01, -1.4747001e+00],
       [-6.4616889e-01,  6.4852601e-01,  7.7405632e-02, ...,
        -3.1655696e-01,  2.6238036e-01, -6.6054714e-01],
       ...,
       [ 3.2611141e-01, -3.4134045e-01,  9.1668588e-01, ...,
        -6.6093838e-01, -7.0826337e-04, -1.0672311e+00],
       [ 4.1295210e-01, -4.7253543e-01,  9.3166596e-01, ...,
        -6.8352866e-01, -1.8768054e-02, -1.0531371e+00],
       [ 4.5918289e-01, -5.2467126e-01,  7.8476018e-01, ...,
        -6.9273007e-01, -9.4007984e-02, -1.3676246e+00]], dtype=float32)

## Try detokenization

This just means that the embedding of words like em
##bed
##ding
##s will be the average of the embedding of each part of it